# ***Assignment 9 - word2vec***


In [15]:
#import statements
import string
from sklearn.preprocessing import normalize

In [16]:
# Verify tenseorflow version
%tensorflow_version 1.x

import tensorflow as tf
import numpy as np

tf.__version__

'1.15.2'

In [17]:
# Word pre processing function
def pre_process(word):
  word = word.lower().strip()
  for punc in string.punctuation:
    word = word.replace(punc,"")
  return word

In [18]:
 # input processing function to generate word pairs
 def process_input(filepath): 

  #initialize
  sentences = open(filepath, 'r').readlines()
  size_of_window = 3
  word_pairs = []
  sentence_list = []

  # process input
  for sentence in sentences:
    sentence_list = [pre_process(word) for word in sentence.split()]
    for i, target in enumerate(sentence_list):
      for j in range(1,4):
        if i+j < len(sentence_list):
          word_pairs.append((target,sentence_list[i+j]))
        if i-j>= 0:
          word_pairs.append((target,sentence_list[i-j]))

  # generate word to id, id to word and word pairs
  IdToWord = list(set(pair[0] for pair in word_pairs))
  wordToId = {w:i for i,w in enumerate(IdToWord)}
  word_pairs = [(wordToId[pair[0]],wordToId[pair[1]]) for pair in word_pairs]

  return word_pairs, IdToWord ,wordToId


In [19]:
# Function to train the model 
def train(word_pairs, IdToWord ,wordToId):

  # Initialize
  vocubSize = len(wordToId)
  embeddingSize = 50
  batchSize = 1024

  # weights and embeddings
  w_embed = tf.Variable(tf.random_uniform([vocubSize,embeddingSize]))
  weights = tf.Variable(tf.random_uniform([embeddingSize,vocubSize]))

  #input and output data
  batch = tf.placeholder(tf.int32)
  x_train = batch[:,0]
  y_train = batch[:,1]

  # prediction
  wembed = tf.nn.embedding_lookup(w_embed,x_train)
  prediction = tf.matmul(wembed,weights)

  #optimizer
  loss = tf.losses.sparse_softmax_cross_entropy(y_train, prediction)
  optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

  init=tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)

    #Training loop
    for i in range(0, len(word_pairs), batchSize):
      batchData = word_pairs[i : i + batchSize]
      _,lossValue = sess.run([optimizer,loss],feed_dict={batch : batchData})

    W = sess.run(w_embed)
    return W

In [24]:
# Function to get word top `k` similarities
def similarity(word, word_to_id, id_to_word, W_, k):
  vec = W_[word_to_id[word.lower()]]
  sim = np.dot(W_,vec)
  sim_idx=np.argsort(sim)[::-1]
  res = []
  for idx in sim_idx[1:k+1]:
    res.append((id_to_word[idx],sim[idx]))
  return res

In [22]:
# Training model for a text file fetched from gutenberg files
filepath_gutenberg = './gutenberg.txt'
word_pairs_guten, IdToWord_guten, wordToId_guten = process_input(filepath_gutenberg)
W_guten = train(word_pairs_guten, IdToWord_guten ,wordToId_guten)
np.save('W_gutenberg.npy',W_guten)

In [38]:
# Running the trained model for a text file fetched from gutenberg files
W_gutenberg = np.load("W_gutenberg.npy")
W_gutenberg = normalize(W_gutenberg)
k = 6
word="compliant"
print("Word => {}\nWord similarities => {}\n".format(word, similarity(word, wordToId_guten, IdToWord_guten, W_gutenberg, k)))

word="unattempted"
print("Word => {}\nWord similarities => {}\n".format(word, similarity(word, wordToId_guten, IdToWord_guten, W_gutenberg, k)))

word="glorious"
print("Word => {}\nWord similarities => {}\n".format(word, similarity(word, wordToId_guten, IdToWord_guten, W_gutenberg, k)))

word="cash"
print("Word => {}\nWord similarities => {}\n".format(word, similarity(word, wordToId_guten, IdToWord_guten, W_gutenberg, k)))

word="stripes"
print("Word => {}\nWord similarities => {}\n".format(word, similarity(word, wordToId_guten, IdToWord_guten, W_gutenberg, k)))

Word => compliant
Word similarities => [('engendering', 0.86162275), ('including', 0.85579246), ('syrtis', 0.85484564), ('assures', 0.8493416), ('bitterness', 0.8468277), ('hospitable', 0.84678)]

Word => unattempted
Word similarities => [('withstands', 0.8788162), ('confer', 0.8664285), ('altern', 0.8623103), ('venturous', 0.85828805), ('springtime', 0.8579355), ('sworn', 0.85767287)]

Word => glorious
Word similarities => [('willinger', 0.84164786), ('encroached', 0.83873343), ('tookst', 0.8321131), ('timbrels', 0.8312454), ('adversaryserpent', 0.8309327), ('trance', 0.8292624)]

Word => cash
Word similarities => [('frisking', 0.87216794), ('whateer', 0.86321104), ('him', 0.83330226), ('hope', 0.8330941), ('he', 0.82983404), ('offerings', 0.82980025)]

Word => stripes
Word similarities => [('etext', 0.88781816), ('needest', 0.8673834), ('who', 0.86489224), ('anchors', 0.86488086), ('rebuke', 0.85766), ('swage', 0.8573093)]

